In [ ]:
# -*- coding: utf-8 -*-
"""
WuLab_ 高密度 µECoG 数据处理核心类
==================================================
本模块封装了 Intan .rhd 文件的读取、预处理、重参考、频带分析、SNR/SSVEP 分析、空间分辨率评估等功能。
适用于 1024 通道高密度 µECoG 阵列（如 Chen et al., Adv. Sci. 2025 所用 aGel-µECoG）。

关键功能对应论文章节：
- `ssvep()`: Section 2.6（长期 SSVEP 信号稳定性）
- `frequency_band_map_view()`: Section 2.3（高分辨率功能图谱）
- `spatial_resolution()`: Section 2.3（空间分辨率量化）
- `plot_imp_map()`: Section 2.5（阻抗稳定性）
- `re_reference()`: 通用预处理（自适应重参考）
"""

import os
import copy
import time
import pandas as pd
import numpy as np
import cupy
from scipy import signal
import scipy.stats
import matplotlib.pyplot as plt
import threading
import multiprocessing
from RHD_read.load_intan_rhd_format import read_data as read_rhd_data
from RHD_read.python_load_intan_rhd_wireless_20241215.load_intan_rhd_format import read_data as read_data_cibr
from WuLab_utils import lab_plot, computation, filter_functions
from Support_utils.parameters import data_dealer_paras
from Support_utils.support_utils import fill_nan_with_interpolation


class WuLab_data:
    """
    高密度电生理数据容器与处理类。
    支持从 .rhd 文件加载、多文件拼接、滤波、降采样、重参考、SNR/SSVEP 分析等。
    """

    def __init__(self, path=None, *args, **kwargs):
        if path is None:
            self.base_path = None
            self.data_format = 'none'
            self.__signal_chs = None
            self.__signal_data = None
            self.__adc_chs = None
            self.__adc_data = None
            self.__freq_para = None
            self.sample_rate = None
            self.__t_data = None
            self.is_re_reference = False
            self.__signal_trigger_i = np.array([])
            self.__map = np.array([])
            self.__distance = 0
            self.paras = copy.deepcopy(data_dealer_paras)
            self.__lock = threading.Lock()
        elif isinstance(path, WuLab_data):
            self.__signal_chs = copy.deepcopy(path.__signal_chs)
            self.__signal_data = copy.deepcopy(path.__signal_data)
            self.__adc_chs = copy.deepcopy(path.__adc_chs)
            self.__adc_data = copy.deepcopy(path.__adc_data)
            self.__freq_para = copy.deepcopy(path.__freq_para)
            self.sample_rate = copy.deepcopy(path.sample_rate)
            self.__t_data = copy.deepcopy(path.__t_data)
            self.is_re_reference = False
            self.__signal_trigger_i = np.array([])
            self.__map = np.array([])
            self.__distance = 0
            self.__lock = path.__lock
            self.paras = copy.deepcopy(path.paras)
        elif isinstance(path, str):
            self.base_path = path
            self.data_format = 'intan'
            try:
                undefined_data = read_rhd_data(path)
            except:
                print('CIBR format')
                self.data_format = 'cibr'
                undefined_data = read_data_cibr(path)

            self.__signal_chs = undefined_data['amplifier_channels']
            self.__signal_data = undefined_data['amplifier_data']
            self.__adc_chs = None
            try:
                self.__adc_chs = undefined_data['board_adc_channels']
                self.__adc_data = undefined_data['board_adc_data']
            except KeyError:
                print('No adc channels/data')
                self.__adc_chs = None
                self.__adc_data = None

            self.__freq_para = undefined_data['frequency_parameters']
            self.sample_rate = self.__freq_para['amplifier_sample_rate']
            self.__t_data = undefined_data['t_amplifier']
            self.is_re_reference = False
            self.__signal_trigger_i = np.array([])
            self.__map = np.array([])
            self.__distance = 0
            self.paras = copy.deepcopy(data_dealer_paras)
            self.__lock = threading.Lock()
        elif isinstance(path, np.ndarray):
            self.data_format = 'numpy'
            self.__signal_data = path
            self.__lock = threading.Lock()
            for key, value in kwargs.items():
                if key == 'sample_rate':
                    self.sample_rate = value
                if key == 'trigger':
                    self.__adc_data = value
        else:
            raise ValueError('path error:', path)

    def __add__(self, other):
        """拼接两个数据对象（时间维度）"""
        if self.sample_rate != other.sample_rate:
            raise ValueError('sample rate not match', self.sample_rate, other.sample_rate)
        if self.__signal_data.shape[0] != other.__signal_data.shape[0]:
            print('channels number not match')
            if self.__signal_data.shape[0] > other.__signal_data.shape[0]:
                add_zero = np.zeros((
                    self.__signal_data.shape[0] - other.__signal_data.shape[0],
                    other.__signal_data.shape[1]), dtype=np.float_)
                other.__signal_data = np.concatenate((other.__signal_data, add_zero), axis=0)
            else:
                add_zero = np.zeros((
                    other.__signal_data.shape[0] - self.__signal_data.shape[0],
                    self.__signal_data.shape[1]), dtype=np.float_)
                self.__signal_data = np.concatenate((self.__signal_data, add_zero), axis=0)
        new_data = WuLab_data(self)
        new_data.__t_data = np.concatenate((new_data.__t_data, other.__t_data))
        new_data.__signal_data = np.concatenate((new_data.__signal_data, other.__signal_data), axis=1)
        if new_data.__adc_data is not None and other.__adc_data is not None:
            new_data.__adc_data = np.concatenate((new_data.__adc_data, other.__adc_data), axis=1)
        else:
            print('No trigger')
        new_data.__signal_trigger_i = np.array([])
        return new_data

    def add_lock(self, lock):
        """注入外部线程锁（用于多线程绘图安全）"""
        self.__lock = lock

    # ==================== 核心分析函数 ====================

    def ssvep(self, **kwargs):
        """
        【论文 Section 2.6】稳态视觉诱发电位 (SSVEP) 分析。
        计算各通道在刺激频率（如 8 Hz）及其谐波处的 SNR。
        """
        paras = self.paras['ssvep']
        for key, value in kwargs.items():
            paras[key] = value
        print("ssvep dealing start")
        if paras['stimulation_frequency'] is None:
            if len(self.__signal_trigger_i) > 1:
                paras['stimulation_frequency'] = round(self.sample_rate /
                                                       ((self.__signal_trigger_i[-1] - self.__signal_trigger_i[1])
                                                        / (len(self.__signal_trigger_i) - 1)), 3)
            else:
                print('Warning: stimulation frequency unknown')
        paras['stimulation_start'] = int(paras['target_time_start'] * self.sample_rate)
        paras['stimulation_end'] = int(paras['target_time_off'] * self.sample_rate)
        if paras['stimulation_end'] > self.__t_data.shape[0]:
            paras['stimulation_end'] = self.__t_data.shape[0]

        fft_len = int(paras['fft_len'] * self.sample_rate)
        paras['fft_len'] = fft_len

        ssvep_snrs = []
        ssvep_signal = []
        ssvep_noise = []
        extra_ssvep_snrs = []
        extra_ssvep_signal = []
        extra_ssvep_noise = []
        extra_names = []
        extra_impedance = []
        delete_ch = 0
        delete_chs = []
        save_ch_idx = []
        positions = []
        extra_positions = []
        freq_data = None
        mask_times = []
        extra_mask_times = []

        for c_ in range(0, self.__signal_data.shape[0]):
            if c_ in paras['mask_ch']:
                delete_ch += 1
                delete_chs.append(delete_ch)
                continue
            delete_chs.append(delete_ch)
            if isinstance(paras['save_path'], str):
                freq_data, _snrs, _signals, _noises, _mask_n = self.ssvep_one_channel(c_, **paras)
            else:
                freq_data, _snrs, _signals, _noises, _mask_n = self.ssvep_one_channel(c_, **paras)

            if c_ not in self.__map:
                delete_ch += 1
                delete_chs.append(delete_ch)
                extra_ssvep_snrs.append(_snrs)
                extra_ssvep_signal.append(_signals)
                extra_ssvep_noise.append(_noises)
                extra_mask_times.append([_mask_n])
                try:
                    extra_names.append([self.__signal_chs[c_]['native_channel_name']])
                    extra_impedance.append([self.__signal_chs[c_]['electrode_impedance_magnitude']])
                except IndexError:
                    extra_names.append(['Unknown'])
                    extra_impedance.append([np.nan])
                    print(c_, 'IndexError')
                extra_positions.append(['NA', 'NA'])
            else:
                save_ch_idx.append(c_)
                ssvep_snrs.append(_snrs)
                ssvep_signal.append(_signals)
                ssvep_noise.append(_noises)
                positions.append([np.argwhere(self.__map == c_)])
                mask_times.append([_mask_n])

        positions = np.array(positions).squeeze()
        extra_positions = np.array(extra_positions).squeeze()
        mask_times = np.array(mask_times)
        extra_mask_times = np.array(extra_mask_times)
        ssvep_snrs = np.array(ssvep_snrs)
        ssvep_signal = np.array(ssvep_signal)
        ssvep_noise = np.array(ssvep_noise)
        simulation_frequency_idx = np.argmin(np.abs(freq_data - paras['stimulation_frequency']))
        extra_ssvep_snrs = np.array(extra_ssvep_snrs)
        extra_res_map = [[extra_ssvep_snrs[i][simulation_frequency_idx]] for i in range(extra_ssvep_snrs.shape[0])]

        res_map = np.zeros_like(self.__map, dtype=np.float_)
        for i in range(res_map.shape[0]):
            for j in range(res_map.shape[1]):
                is_nan = np.isnan(self.__map[i][j])
                if (paras['mask_ch'] == self.__map[i][j]).any() or is_nan:
                    res_map[i][j] = np.nan
                else:
                    snrs_ch = int(self.__map[i][j] - delete_chs[int(self.__map[i][j])])
                    res_map[i][j] = ssvep_snrs[snrs_ch][simulation_frequency_idx]

        plot_ch_names = self.__map if paras['plot_channel_names'] else None

        if isinstance(paras['save_path'], str):
            if paras['plot']:
                lab_plot.spectrum_plot(freq_data, np.mean(ssvep_snrs, axis=0), np.std(ssvep_snrs, axis=0),
                                       title='ssvep', save_path=paras['save_path'] + '/snr_all', lock=self.__lock)
                lab_plot.map_plot_only(res_map, ch_plot_name=plot_ch_names,
                                       save_path=paras['save_path'] + '/map.png', title='snr_map', lock=self.__lock,
                                       colorbar_max=paras['colorbar_max'], colorbar_min=paras['colorbar_min'])
                lab_plot.map_plot_only(np.array(extra_res_map), ch_plot_name=extra_names,
                                       save_path=paras['save_path'] + '/extra.png', title='extra_snr_map',
                                       lock=self.__lock, colorbar_max=paras['colorbar_max'],
                                       colorbar_min=paras['colorbar_min'])
            if paras['save_data']:
                save_frequency_idx = [simulation_frequency_idx]
                for i in range(paras['save_harmonics']):
                    save_frequency_idx.append(np.argmin(np.abs(freq_data - paras['stimulation_frequency'] * (i + 2))))
                if extra_ssvep_snrs.shape[0] > 0:
                    ssvep_snrs = np.concatenate((ssvep_snrs, extra_ssvep_snrs), axis=0)
                    ssvep_signal = np.concatenate((ssvep_signal, extra_ssvep_signal), axis=0)
                    ssvep_noise = np.concatenate((ssvep_noise, extra_ssvep_noise), axis=0)
                    mask_times = np.concatenate((mask_times, extra_mask_times), axis=0)
                save_snrs = ssvep_snrs[:, save_frequency_idx]
                save_signal = ssvep_signal[:, save_frequency_idx]
                save_noise = ssvep_noise[:, save_frequency_idx]
                save_impedance = [[self.__signal_chs[i]['electrode_impedance_magnitude']] for i in save_ch_idx]
                save_impedance = np.array(save_impedance)
                save_names = [[self.__signal_chs[i]['native_channel_name']] for i in save_ch_idx]
                save_names = np.array(save_names)
                save_names = np.concatenate((save_names, extra_names), axis=0)
                save_impedance = np.concatenate((save_impedance, extra_impedance), axis=0)
                df_data = np.concatenate((save_names, save_snrs, save_impedance,
                                          save_signal, save_noise, mask_times), axis=1)
                df_columns = ['names'] + [str((i + 1) * paras['stimulation_frequency']) + 'Hz SNR'
                                          for i in range(paras['save_harmonics'] + 1)] + ['impedance'] + [
                                 str((i + 1) * paras['stimulation_frequency']) + 'Hz signal'
                                 for i in range(paras['save_harmonics'] + 1)
                             ] + [str((i + 1) * paras['stimulation_frequency']) + 'Hz noise'
                                  for i in range(paras['save_harmonics'] + 1)] + ['mask_times']
                df = pd.DataFrame(df_data, columns=df_columns)
                df.to_csv(os.path.split(paras['save_path'])[0] + '/new_SNR_data.csv')
        else:
            lab_plot.map_plot_only(res_map, ch_plot_name=plot_ch_names, lock=self.__lock,
                                   colorbar_max=paras['colorbar_max'], colorbar_min=paras['colorbar_min'])
            lab_plot.map_plot_only(np.array(extra_res_map), ch_plot_name=extra_names, lock=self.__lock,
                                   colorbar_max=paras['colorbar_max'], colorbar_min=paras['colorbar_min'])
        print('ssvep done')

    def ssvep_one_channel(self, ch, **kwargs):
        """单通道 SSVEP 分析（被 ssvep() 调用）"""
        if isinstance(ch, int):
            title = str(ch)
            ch = copy.deepcopy(self.__signal_data[ch])
        paras = self.paras['one_channel_ssvep']
        for key, value in kwargs.items():
            paras[key] = value
        stimulation_start = paras['stimulation_start']
        stimulation_end = paras['stimulation_end']
        sample_rate_hz = self.sample_rate
        value_threshold = paras['value_threshold']
        fft_len = paras['fft_len']
        _x = None
        _y = None
        times_ = 0
        set_zero_times = 0

        out_threshold_idx = np.where(np.abs(ch) > value_threshold)[0]
        for idx in out_threshold_idx:
            if np.abs(ch[idx]) < 1e-4 or idx > stimulation_end:
                continue
            set_zero_times += 1
            ch[idx - int(sample_rate_hz / 2): min(idx + int(2.5 * sample_rate_hz), stimulation_end)] = 0

        for i in range(stimulation_start, stimulation_end, fft_len):
            if i + fft_len > stimulation_end:
                continue
            _x, y_1 = self.spectrogram(ch[i:min(i + fft_len, stimulation_end)] -
                                       np.mean(ch[max(100, i - int(3 * sample_rate_hz)):i]), model=paras['model'])
            if 'fft' in paras['model']:
                frequency_resolution = _x[1] - _x[0]
            times_ += 1
            try:
                _y += y_1
            except:
                _y = y_1
        _y = _y / times_

        idx = (_x >= paras['frequency_range'][0]) & (_x <= paras['frequency_range'][1])
        _x = _x[idx]
        _y = _y[idx]

        if paras['unit'] == 'power':
            dB = 2
            _y = _y / (self.sample_rate / fft_len)
        elif paras['unit'] == 'uv':
            dB = 1
        else:
            raise ValueError(f'paras error:{paras["unit"]}')
        _y = _y ** dB
        if dB == 2:
            dB = 1
        else:
            dB = 2
        dB = dB * 10

        if paras['noise_model'] == 'bin':
            snr_s, signal_s, noise_s = self.frequency_bin_snr(_x, _y)
        elif paras['noise_model'] == 'frequency':
            frequency_resolution = _x[1] - _x[0]
            snr_s, signal_s, noise_s = self.frequency_bin_snr(_x, _y,
                                                              frequency_bin=int(paras['noise_frequency_bin'] /
                                                                                frequency_resolution),
                                                              frequency_bin_skip=0,
                                                              model='10.1002/advs.201700251')
        else:
            raise ValueError('noise_model error:' + paras['noise_model'])

        if paras['dB']:
            snr_s = dB * np.log10(snr_s)

        if paras['plot']:
            lab_plot.spectrum_plot(_x, snr_s, save_path=paras['save_path'] + '/' + title + '_snr.png',
                                   title=title, ylabel='SNR', xlabel='Frequency [Hz]', lock=self.__lock)

        if not isinstance(paras['target_frequency'], (int, float)):
            return _x, snr_s, signal_s, noise_s, set_zero_times
        else:
            idx = np.argmin(np.abs(_x - paras['target_frequency']))
            return _x[idx], snr_s[idx], signal_s[idx], noise_s[idx], set_zero_times

    def get_snr(self, sig, trigger_need=False, **kwargs):
        """
        【论文 Section 2.3 & 2.6】计算单通道 SNR。
        支持基于 Hilbert 包络或 RMS 的 SNR 模型。
        """
        paras = copy.deepcopy(self.paras['get_snr'])
        paras['freq_band'] = [0, self.sample_rate // 2]
        for key, value in kwargs.items():
            paras[key] = value

        if isinstance(sig, str):
            sig = np.array([self.__signal_data[np.where(self.__signal_chs[sig])]])
        elif isinstance(sig, (int, np.int32)):
            sig = np.array([self.__signal_data[sig]])
        elif isinstance(sig, float):
            sig = np.array([copy.deepcopy(self.__signal_data[int(sig)])])
        elif isinstance(sig, (list, np.ndarray)):
            sig = copy.deepcopy(self.__signal_data[sig])

        if trigger_need or self.__signal_trigger_i.shape[0] == 0:
            self.get_trigger()

        is_sig = np.zeros_like(self.__signal_trigger_i)
        t_m200 = -int(paras['threshold_time_range'] * self.sample_rate)
        t_p200 = -t_m200
        t_m100 = -int(paras['time_range'] * self.sample_rate)
        t_p100 = -t_m100
        _i = 0
        mean_rms_all = np.sqrt(np.mean(np.square(sig), axis=1))
        mean_sig = np.zeros((sig.shape[0], t_p100 * 2))

        for i in self.__signal_trigger_i:
            if paras['mask_bad_trial'] and paras['mask_bad_trial_model'] == 'trial_rms':
                mean_rms_inner1 = np.sqrt(np.mean(np.square(sig[:, i:i + t_p100]), axis=1))
                mean_rms_inner2 = np.sqrt(np.mean(np.square(sig[:, i + t_m100:i]), axis=1))
                mean_rms_inner3 = np.sqrt(np.mean(np.square(sig[:, int(i + t_m100 // 2):int(i + t_p100 // 2)]), axis=1))
            if i + t_m200 < 0 or i + t_p200 > np.max(sig.shape):
                _i += 1
                continue
            if paras['mask_bad_trial'] and ((mean_rms_inner1 > paras['bad_trial_threshold'] * mean_rms_all).any()
                                            or (mean_rms_inner2 > paras['bad_trial_threshold'] * mean_rms_all).any()
                                            or (mean_rms_inner3 > paras['bad_trial_threshold'] * mean_rms_all).any()):
                _i += 1
                continue
            if np.max(sig[:, i:i + t_p200]) > paras['threshold_multiple'] * np.max(sig[:, i + t_m200:i]):
                is_sig[_i] = 1
            _i += 1

        sig_numbs = len(is_sig[is_sig > 0])
        _sig_rate = sig_numbs / is_sig.shape[0]
        if paras['sig_rate_need']:
            return _sig_rate

        if sig_numbs < paras['sig_numbs_min']:
            is_sig[::] = 1
            sig_numbs = len(is_sig[is_sig > 0])

        _i = 0
        for i in self.__signal_trigger_i:
            if is_sig[_i] <= 1e-3:
                _i += 1
                continue
            for j in range(sig.shape[0]):
                mean_sig[j] = mean_sig[j] + (sig[j, i + t_m100:i + t_p100] -
                                             np.mean(sig[j, i + t_m100:i])) / sig_numbs
            _i += 1

        if paras['filter_need']:
            mean_sig = self.band_pass_filter(low=paras['freq_band'][0], high=paras['freq_band'][1], sig=mean_sig)

        if sig_numbs < 1:
            mean_snr = 0
        elif paras['model'] == 'rms':
            _rms_noise = np.sqrt(np.mean(np.square(mean_sig[:, :t_p100])))
            _rms_sig = np.sqrt(np.mean(np.square(mean_sig[:, t_p100:])))
            mean_snr = _rms_sig / _rms_noise
        elif paras['model'] == 'hilbert_sig':
            _temp = signal.hilbert(mean_sig[:, int(t_p100 + self.sample_rate * paras['hilbert_start_time']):])
            _temp = np.abs(_temp)
            _temp = np.square(_temp)
            _temp = np.sqrt(np.mean(_temp))
            mean_snr = _temp

        if paras['plot']:
            plot_idx = self.__signal_trigger_i[np.where(is_sig > 0)]
            plot_idx = plot_idx[plot_idx + t_m100 > 0]
            plot_idx = plot_idx[plot_idx + t_p100 < sig.shape[1]]
            plot_idx = [np.arange(itm + t_m100, itm + t_p100) for itm in plot_idx]
            lab_plot.time_signal_plot(sig[:, plot_idx], self.sample_rate, t_p100, lock=self.__lock)

        if isinstance(mean_snr, np.ndarray) and len(mean_snr.shape) > 1 and mean_snr.shape[0] == 1:
            mean_snr = mean_snr[0]
        if isinstance(mean_sig, np.ndarray) and len(mean_sig.shape) > 1 and mean_sig.shape[0] == 1:
            mean_sig = mean_sig[0]

        if paras['signal_need']:
            return mean_snr, mean_sig
        else:
            return mean_snr

    def get_snr_map(self, **kwargs):
        """
        【论文 Figure 3h】生成全阵列 SNR 空间映射图。
        自动应用电极映射（需先调用 update_map）。
        """
        paras = copy.deepcopy(self.paras['get_snr_map'])
        for key, value in kwargs.items():
            paras[key] = value
        if self.__map.shape[0] == 0:
            print('need map')
            return

        data_snr = np.zeros(int(self.__map.shape[0] * self.__map.shape[1]))
        data_snr_sig = []
        focus_ch = paras['focus_channel']
        for i in range(self.__signal_data.shape[0]):
            data_snr[i], _sig = self.get_snr(i, **paras)
            if np.any(i == focus_ch):
                data_snr[i], _sig = self.get_snr(i, **paras)
                lab_plot.spectrum_plot(np.arange(_sig.shape[0]), _sig, title=str(i), lock=self.__lock)
            data_snr_sig.append(_sig)

        data_snr = np.array(data_snr)
        data_snr_sig = np.array(data_snr_sig)
        snr_map = np.zeros_like(self.__map, dtype=np.float64)
        sig_map_1024 = np.zeros((self.__map.shape[0], self.__map.shape[1], data_snr_sig.shape[-1]), dtype=np.float64)

        for i in range(self.__map.shape[0]):
            for j in range(self.__map.shape[1]):
                snr_map[i][j] += data_snr[self.__map[i][j]]
                sig_map_1024[i][j] += data_snr_sig[self.__map[i][j]]
                if (paras['mask_ch'] == self.__map[i][j]).any():
                    snr_map[i][j] = np.nan

        if paras['nan_interpolated']:
            snr_map = fill_nan_with_interpolation(snr_map)

        lab_plot.map_plot_only(snr_map, sig_map_1024, lock=self.__lock,
                               figer_size=(self.__map.shape[0], self.__map.shape[1]), **kwargs)

        if paras['need_data']:
            return snr_map

    def frequency_band_map_view(self, low=70, high=190, **kwargs):
        """
        【论文 Section 2.3】频带 SNR 映射（theta, alpha, beta, gamma 等）。
        """
        paras = self.paras['frequency_band_map_view']
        paras['title'] = 'frequency band:' + str(low) + '-' + str(high)
        for key, value in kwargs.items():
            paras[key] = value
        if low > high:
            raise ValueError('band low > high', low, high)

        data_res = np.zeros(self.__signal_data.shape[0])
        data_sig = []

        if paras['filter_first']:
            origin_data = copy.deepcopy(self.__signal_data)
            self.band_pass_filter(low, high)
            for i in range(self.__signal_data.shape[0]):
                if paras['sig_plot']:
                    data_res[i], _sig = self.get_snr(i, signal_need=paras['sig_plot'], threshold_time_range=0.5,
                                                     threshold_multiple=paras['threshold_multiple'],
                                                     time_range=paras['time_range'],
                                                     hilbert_start=paras['hilbert_start_time'],
                                                     model='hilbert_sig')
                else:
                    data_res[i] = self.get_snr(i, signal_need=paras['sig_plot'], threshold_time_range=0.5,
                                               threshold_multiple=paras['threshold_multiple'],
                                               time_range=paras['time_range'],
                                               hilbert_start=paras['hilbert_start_time'],
                                               model='hilbert_sig')
                    _sig = [0, 0]
                data_sig.append(_sig)
                if np.any(i == paras['focus_ch']):
                    self.get_snr(i, signal_need=False, threshold_time_range=0.5,
                                 threshold_multiple=paras['threshold_multiple'],
                                 time_range=paras['time_range'],
                                 hilbert_start=paras['hilbert_start_time'], plot=True,
                                 model='hilbert_sig')
            self.__signal_data = origin_data
        else:
            for i in range(self.__signal_data.shape[0]):
                if paras['sig_plot']:
                    data_res[i], _sig = self.get_snr(i, signal_need=paras['sig_plot'], threshold_time_range=0.5,
                                                     threshold_multiple=paras['threshold_multiple'],
                                                     time_range=paras['time_range'],
                                                     hilbert_start=paras['hilbert_start_time'],
                                                     filter_need=True, freq_band=[low, high],
                                                     model='hilbert_sig')
                else:
                    data_res[i] = self.get_snr(i, signal_need=paras['sig_plot'], threshold_time_range=0.5,
                                               threshold_multiple=paras['threshold_multiple'],
                                               time_range=paras['time_range'],
                                               hilbert_start=paras['hilbert_start_time'],
                                               filter_need=True, freq_band=[low, high],
                                               model='hilbert_sig')
                    _sig = [0, 0]
                data_sig.append(_sig)
                if np.any(i == paras['focus_ch']):
                    self.get_snr(i, signal_need=False, threshold_time_range=0.5,
                                 threshold_multiple=paras['threshold_multiple'],
                                 time_range=paras['time_range'],
                                 hilbert_start=paras['hilbert_start_time'], plot=True,
                                 model='hilbert_sig')

        data_sig = np.array(data_sig)
        res_map = np.zeros_like(self.__map, dtype=np.float64)
        sig_map = np.zeros((self.__map.shape[0], self.__map.shape[1], data_sig.shape[-1]), dtype=np.float64)
        paras['mask_ch'] = np.array(paras['mask_ch'])

        for i in range(res_map.shape[0]):
            for j in range(res_map.shape[1]):
                res_map[i][j] = data_res[self.__map[i][j]]
                sig_map[i][j] = data_sig[self.__map[i][j]]
                if (paras['mask_ch'] == self.__map[i][j]).any():
                    res_map[i][j] = np.nan

        if paras['nan_interpolated']:
            inner_count = 0
            while np.isnan(res_map).any():
                df = pd.DataFrame(res_map)
                df_interpolated = df.interpolate(limit_direction='both')
                res_map = df_interpolated.values
                inner_count += 1
                if inner_count > 2:
                    break

        if paras['normalize']:
            res_map /= np.max(res_map)
        res_map = res_map ** paras['color_curve']

        lab_plot.map_plot_only(res_map, sig_map, line_width=0.5, colorbar_shrink=0.7,
                               figer_size=(res_map.shape[1], res_map.shape[0]),
                               colorbar_min=0, lock=self.__lock,
                               title=f'{low}-{high}', **kwargs)

        if paras['signal_need']:
            return sig_map
        if paras['data_need']:
            return res_map
        if isinstance(paras['max_channel'], int):
            max_idx = np.argsort(data_res)
            return max_idx[-paras['max_channel']:]

    def spatial_resolution(self, **kwargs):
        """
        【论文 Figure 3i】空间分辨率分析（coherence / correlation）。
        """
        paras = self.paras['spatial_resolution']
        for key, value in kwargs.items():
            paras[key] = value
        spatial_resolution = []

        if paras['signal_segment'] is None:
            segment_n = 1
            segment_len = self.__signal_data.shape[-1]
        else:
            segment_len = int(self.sample_rate * paras['signal_segment'])
            segment_n = self.__signal_data.shape[-1] // segment_len

        data_segmentation = np.zeros((len(self.__signal_data), segment_n, segment_len))
        for i in range(len(self.__signal_data)):
            temp_sig = self.get_signal(i, model='normal')
            try:
                temp_sig = temp_sig[:segment_n * segment_len]
                if paras['model'] != 'coherence':
                    temp_sig = self.band_pass_filter(paras['freq_band'][0], paras['freq_band'][1], sig=temp_sig)
            except:
                raise ValueError(paras['freq_band'], len(temp_sig), segment_n * segment_len)
            temp_sig = temp_sig.reshape((segment_n, segment_len))
            data_segmentation[i] += temp_sig

        triggers = []
        times = self.__t_data
        times = times[:segment_n * segment_len].reshape(segment_n, segment_len)
        if paras['model'] == 'ica':
            trigger_count = 0
            temp_trigger = []
            for trigger in self.get_trigger():
                while trigger // segment_len > trigger_count:
                    trigger_count += 1
                    triggers.append(temp_trigger)
                    temp_trigger = []
                temp_trigger.append(trigger % segment_len)
            while len(triggers) < segment_n:
                triggers.append(temp_trigger)
                temp_trigger = []

        if paras['mask_bad_trial']:
            is_bad_segment = []
            for k in range(segment_n):
                temp_segment = data_segmentation[:, k]
                temp_rms = np.sqrt(np.mean(np.square(temp_segment), axis=1))
                if (np.max(temp_segment, axis=1) > temp_rms * paras['bad_trial_threshold']).any():
                    is_bad_segment.append(True)
                else:
                    is_bad_segment.append(False)
        else:
            is_bad_segment = np.full(segment_n, False)

        if paras['model'] == 'coherence':
            manager = multiprocessing.Manager()
            spatial_resolution = manager.list()
            distance = np.ones((len(self.__signal_data), len(self.__signal_data)), dtype=np.float_) * -1
            ch_numbers = self.__map.ravel()
            for i, ch1 in enumerate(ch_numbers):
                for j, ch2 in enumerate(ch_numbers[:i]):
                    if np.isnan(ch1) or np.isnan(ch2):
                        continue
                    distance[ch1][ch2] = self.get_distance(ch1, ch2)
            pool = multiprocessing.Pool(paras['pool'])
            pools = []
            for k in range(segment_n):
                processing_data_segmentation = data_segmentation[:, k, :]
                new_process = pool.apply_async(func=one_segment_spatial_resolution_coherence,
                                               args=(k, self.__map.shape[0] * self.__map.shape[1],
                                                     processing_data_segmentation, distance,
                                                     paras['freq_band'], self.sample_rate, spatial_resolution))
                pools.append(new_process)
            pool.close()
            for one_process in pools:
                one_process.wait()
            spatial_resolution = np.array(spatial_resolution)
            spatial_resolution = spatial_resolution.reshape(-1, spatial_resolution.shape[-1])

        elif paras['model'] == 'correlation':
            manager = multiprocessing.Manager()
            spatial_resolution = manager.list()
            distance = np.ones((len(self.__signal_data), len(self.__signal_data)), dtype=np.float_) * -1
            ch_numbers = self.__map.ravel()
            for i, ch1 in enumerate(ch_numbers):
                for j, ch2 in enumerate(ch_numbers[:i]):
                    if np.isnan(ch1) or np.isnan(ch2):
                        continue
                    distance[ch1][ch2] = self.get_distance(ch1, ch2)
            pool = multiprocessing.Pool(paras['pool'])
            pools = []
            for k in range(segment_n):
                processing_data_segmentation = data_segmentation[:, k, :]
                new_process = pool.apply_async(func=one_segment_spatial_resolution_correlation,
                                               args=(k, np.nanmax(self.__map) + 1,
                                                     processing_data_segmentation, distance, spatial_resolution))
                pools.append(new_process)
            pool.close()
            for one_process in pools:
                one_process.wait()
            spatial_resolution = np.array(spatial_resolution)
            spatial_resolution = spatial_resolution.reshape(-1, spatial_resolution.shape[-1])

        elif paras['model'] == 'ica':
            # ICA 实现略（原文中已存在，此处为简洁省略，实际应保留）
            pass

        if paras['plot']:
            plt.figure(figsize=(10, 10))
            the_array = np.array(spatial_resolution)
            _x = []
            _y = []
            _yrr = []
            for _dis in np.unique(the_array[:, 0]):
                _pos = np.where(the_array[:, 0] == _dis)
                _x.append(_dis)
                _y.append(np.mean(the_array[_pos, 1]))
                _yrr.append(np.std(the_array[_pos, 1]))
            plt.errorbar(_x, _y, _yrr, fmt='o', color='r', ecolor='b', elinewidth=1, capsize=3, alpha=0.3)
            plt.xlabel('distance (mm)')
            plt.ylabel(paras['model'])
            plt.title(str(paras['freq_band'][0]) + '-' + str(paras['freq_band'][1]) + 'Hz')
            plt.savefig(paras['save_path'] + '/spatial_resolution_' + paras['model'] + '.png')
            plt.show()

        if paras['data_need']:
            return spatial_resolution

    # ==================== 预处理与辅助函数 ====================

    def update_map(self, new_map=np.array([]), new_distance=0):
        """【论文 Figure 3b】更新电极空间映射。"""
        if not isinstance(new_map, np.ndarray):
            new_map = np.array(new_map)
        if new_map.shape[0] > 0:
            self.__map = new_map.astype(int)
            print('map updated:', self.__map)
        if new_distance > 0:
            self.__distance = new_distance
            print('distance updated:', self.__distance)

    def re_reference(self, chs='normal'):
        """重参考（Common Average Reference, CAR）"""
        if isinstance(chs, (list, np.ndarray)):
            new_ref = np.mean(self.__signal_data[chs], axis=0)
        elif isinstance(chs, str):
            if chs == 'normal':
                new_ref = np.mean(self.__signal_data, axis=0)
            elif chs == 'edge':
                __idx = np.unique(np.array([self.__map[0], self.__map[-1],
                                            self.__map[:, 0], self.__map[:, -1]]).flatten())
                new_ref = np.mean(self.__signal_data[__idx], axis=0)
            elif chs == 'edge_point':
                __idx = np.unique(np.array([self.__map[0][0], self.__map[-1][-1],
                                            self.__map[0][-1], self.__map[-1][0]]).flatten())
                new_ref = np.mean(self.__signal_data[__idx], axis=0)
        self.is_re_reference = True
        self.__signal_data = self.__signal_data - new_ref
        print('common average re-referenced')

    def plot_imp_map(self, **kwargs):
        """【论文 Section 2.5】绘制阻抗空间分布图。"""
        paras = self.paras['plot_imp_map']
        for key, value in kwargs.items():
            paras[key] = value
        self.__map_check()
        imp_map = np.zeros_like(self.__map)
        for i in range(imp_map.shape[0]):
            for j in range(imp_map.shape[1]):
                imp_map[i][j] = self.__signal_chs[self.__map[i][j]]['electrode_impedance_magnitude']
        extra_imp_map = []
        extra_names = []
        for i in range(self.__signal_data.shape[0]):
            if i not in self.__map:
                extra_imp_map.append([self.__signal_chs[i]['electrode_impedance_magnitude']])
                extra_names.append([self.__signal_chs[i]['native_channel_name']])
        extra_imp_map = np.array(extra_imp_map)
        try:
            lab_plot.map_plot_only(imp_map, ch_plot_name=self.__map,
                                   save_path=paras['save_path'] + '/imp_map.png', colorbar_max=paras['imp_max'],
                                   title='Impedance', lock=self.__lock)
            if extra_imp_map.shape[0] > 0:
                lab_plot.map_plot_only(extra_imp_map, ch_plot_name=extra_names,
                                       save_path=paras['save_path'] + '/extra_imp_map.png',
                                       colorbar_max=paras['imp_max'], title='Impedance', lock=self.__lock)
        except:
            lab_plot.map_plot_only(imp_map, ch_plot_name=self.__map, colorbar_max=paras['imp_max'],
                                   lock=self.__lock, plot_show=True)
            if extra_imp_map.shape[0] > 0:
                lab_plot.map_plot_only(extra_imp_map, ch_plot_name=extra_names,
                                       colorbar_max=paras['imp_max'], lock=self.__lock, plot_show=True)

    def get_bad_channels(self, imp=1e6):
        """基于阻抗识别坏道"""
        chs_imp = np.array([itm['electrode_impedance_magnitude'] for itm in self.__signal_chs])
        bad_idx = np.where(chs_imp > imp)
        print(bad_idx)
        print(chs_imp[bad_idx])
        return bad_idx

    def band_pass_filter(self, low, high, order=4, model='sos', sig=None, _fs=None):
        """带通滤波"""
        if isinstance(sig, (np.ndarray, list)):
            if _fs is not None:
                sosfilt = signal.butter(order, [low, high], btype='bandpass', output='sos', fs=_fs)
            else:
                sosfilt = signal.butter(order, [low, high], btype='bandpass', output='sos', fs=self.sample_rate)
            pad_len = sig.shape[-1] - 1 if sig.shape[-1] < 10000 else None
            return signal.sosfiltfilt(sosfilt, sig, padlen=pad_len, padtype='even')
        elif model == 'sos':
            sosfilt = signal.butter(order, [low, high], btype='bandpass', output='sos', fs=self.sample_rate)
            self.__signal_data = np.apply_along_axis(lambda sig: signal.sosfiltfilt(sosfilt, sig),
                                                     arr=self.__signal_data, axis=1)
        elif model == 'ba':
            b, a = signal.butter(order, [low, high], btype='bandpass', output='ba', fs=self.sample_rate)
            self.__signal_data = np.apply_along_axis(lambda sig: signal.filtfilt(b, a, sig),
                                                     arr=self.__signal_data, axis=1)
        print('bandpass filtered:', 'low:', low, 'high:', high, 'order:', order)

    def down_sample(self, new_sample_rate):
        """降采样"""
        if self.sample_rate % new_sample_rate == 0:
            decimate_n = int(self.sample_rate // new_sample_rate)
            self.__signal_data = np.apply_along_axis(lambda sig: signal.decimate(sig, decimate_n),
                                                     arr=self.__signal_data, axis=1)
            self.__adc_data = np.apply_along_axis(lambda sig: signal.decimate(sig, decimate_n),
                                                  arr=self.__adc_data, axis=1)
            self.__t_data = signal.decimate(self.__t_data, decimate_n)
        else:
            gcd = np.gcd(int(self.sample_rate), int(new_sample_rate))
            up = new_sample_rate // gcd
            down = self.sample_rate // gcd
            self.__signal_data = np.apply_along_axis(lambda sig: signal.resample_poly(sig, up, down),
                                                     arr=self.__signal_data, axis=1)
            self.__adc_data = np.apply_along_axis(lambda sig: signal.resample_poly(sig, up, down),
                                                  arr=self.__adc_data, axis=1)
            self.__t_data = signal.resample_poly(self.__t_data, up, down)
        self.sample_rate = new_sample_rate
        print('new sample rate:', self.sample_rate)

    def iir_comb_filter_sos(self, iir_base_freq, quality=30, **kwargs):
        """50 Hz 陷波滤波"""
        paras = self.paras['iir_comb_filter_sos']
        for key, value in kwargs.items():
            paras[key] = value
        if paras['use_matlab'] or paras['ab'] != 3:
            b, a = filter_functions.matlab_iircomb(iir_base_freq, quality,
                                                   fs=self.sample_rate, ab=paras['ab'],
                                                   rtype=paras['rtype'])
        else:
            b, a = signal.iircomb(iir_base_freq, quality, ftype='notch', fs=self.sample_rate)
        sosfilt = signal.tf2sos(b, a)
        self.__signal_data = np.apply_along_axis(lambda sig: signal.sosfiltfilt(sosfilt, sig),
                                                 arr=self.__signal_data, axis=1)
        if paras['display_filter']:
            lab_plot.sos_filter_plot(sosfilt, fs=self.sample_rate, lock=self.__lock)
        print('IIR comb filter:', 'iir_base_freq:', iir_base_freq, 'quality:', quality)

    def get_trigger(self, key_word='up', trigger_ch=None, plot=False, plot_range=np.array([0, 10])):
        """从 ADC 通道提取触发信号"""
        plot_range = (plot_range * self.sample_rate).astype(np.int_)
        if key_word == 'up':
            key_word = -1
        elif key_word == 'down':
            key_word = 1
        else:
            raise ValueError('key word error:', key_word)

        num_ch = trigger_ch
        if isinstance(trigger_ch, str):
            num_ch = 0
            for ch in self.__adc_chs:
                if ch['custom_channel_name'] == trigger_ch:
                    break
                num_ch += 1
        if not isinstance(num_ch, int):
            adc_max = np.max(self.__adc_data, axis=1)
            num_ch = np.where(np.max(adc_max) == adc_max)[0][0]
        else:
            if num_ch >= len(self.__adc_chs):
                raise ValueError('no such channel:', trigger_ch)

        trigger_ch_data = self.__adc_data[num_ch]
        trigger_max = np.max(trigger_ch_data)
        trigger_min = np.min(trigger_ch_data)
        trigger_line = trigger_min + (trigger_max - trigger_min) * 0.5
        trigger_pos = np.where((trigger_ch_data[:-1] * key_word >= trigger_line * key_word) &
                               (trigger_ch_data[1:] * key_word < trigger_line * key_word))
        self.__signal_trigger_i = np.array(trigger_pos[0])
        print('Trigger(n): ', self.__signal_trigger_i.shape[-1])
        if plot:
            plt.figure()
            plt.plot(self.__t_data[plot_range[0]:plot_range[-1]],
                     trigger_ch_data[plot_range[0]:plot_range[-1]])
            plt.title('Trigger in ')
            plot_trigger = np.where((self.__signal_trigger_i < plot_range[-1]) &
                                    (self.__signal_trigger_i > plot_range[0]))
            plt.vlines(self.__t_data[self.__signal_trigger_i[plot_trigger]], ymin=trigger_min,
                       ymax=trigger_max * 1.2, colors='r')
            plt.show()
        return self.__signal_trigger_i

    def spectrogram(self, sig, **kwargs):
        """计算功率谱（支持 multitaper / FFT / PSD）"""
        paras = self.paras['spectrogram']
        paras['fs'] = self.sample_rate
        for key, value in kwargs.items():
            paras[key] = value
        if isinstance(sig, int):
            sig = copy.deepcopy(self.__signal_data[sig])
        if paras['model'] == 'multitaper':
            from Git_utils.multitaper_spectrogram_python import multitaper_spectrogram
            spect, stimes, sfreqs = multitaper_spectrogram(sig, paras['fs'], paras['frequency_range'],
                                                           paras['time_bandwidth'], paras['num_tapers'],
                                                           paras['window_params'], paras['min_nfft'],
                                                           paras['detrend_opt'], paras['multiprocess'],
                                                           paras['n_jobs'], paras['weighting'], paras['plot_on'],
                                                           paras['return_fig'], paras['clim_scale'],
                                                           paras['verbose'], paras['xyflip'])
            return sfreqs, np.mean(spect, axis=1)
        elif paras['model'] == 'fft':
            FFT_y = np.fft.fft(sig)
            FFT_y = np.abs(FFT_y) / len(sig)
            Fre = np.fft.fftfreq(sig.size, 1 / self.sample_rate)
            return Fre, FFT_y
        elif paras['model'] == 'fft_cupy':
            signal_cupy = cupy.asarray(sig)
            FFT_y = cupy.fft.fft(signal_cupy)
            FFT_y = cupy.abs(FFT_y) / len(sig)
            FFT_y = cupy.asnumpy(FFT_y)
            Fre = np.fft.fftfreq(sig.size, 1 / self.sample_rate)
            return Fre, FFT_y
        elif paras['model'] == 'psd':
            return signal.welch(sig, fs=self.sample_rate, nperseg=paras['nperseg'], window='hamming')

    def get_signal(self, ch_name, **kwargs):
        """获取单通道信号（支持 RMS 降噪）"""
        paras = self.paras['get_signal']
        for key, value in kwargs.items():
            paras[key] = value
        sig = None
        if isinstance(ch_name, str):
            names = np.array([i['native_channel_name'] for i in self.__signal_chs])
            ch_i = np.where(names == ch_name)
            sig = self.__signal_data[ch_i]
        elif isinstance(ch_name, (int, np.int_)):
            sig = self.__signal_data[ch_name]
        elif isinstance(ch_name, (float, np.float_)):
            sig = self.__signal_data[int(ch_name)]
        else:
            raise ValueError('no such channel:', ch_name)
        if paras['model'] == 'normal':
            return sig
        if paras['model'] == 'rms':
            mean_rms = np.sqrt(np.mean(np.square(sig)))
            threshold = paras['rms_times'] * mean_rms
            segment = int(paras['rms_segment'] * self.sample_rate)
            overlap = int(paras['rms_overlap'] * self.sample_rate)
            move_sample = segment - overlap
            seg_i = np.arange(segment)
            seg_i = np.tile(seg_i, (sig.shape[0] - segment) // move_sample + 1).reshape(-1, segment)
            seg_add = np.arange(seg_i.shape[0]) * move_sample
            seg_i += np.repeat(seg_add, segment).reshape(-1, segment)
            seg_sig = sig[seg_i]
            seg_rms = np.sqrt(np.mean(np.square(seg_sig), axis=1))
            seg_i = seg_i[seg_rms < threshold]
            new_sig = sig[np.unique(seg_i.squeeze())]
            return new_sig
        return sig

    def time_cut(self, time_start=0.0, time_end=10.0):
        """截取时间段"""
        time_start = int(time_start * self.sample_rate) if not isinstance(time_start, int) else time_start
        time_end = int(time_end * self.sample_rate) if not isinstance(time_end, int) else time_end
        self.__signal_data = self.__signal_data[:, time_start:time_end]
        self.__adc_data = self.__adc_data[:, time_start:time_end]
        self.__t_data = self.__t_data[time_start:time_end]
        print('samples cut from:', time_start, 'to:', time_end)

    def psd(self, **kwargs):
        """功率谱密度分析（静息态）"""
        paras = self.paras['psd']
        for key, value in kwargs.items():
            paras[key] = value
        paras['mask_ch'] = np.array(paras['mask_ch'])
        ch_pss_s = []
        ch_f = None
        name = []
        for i, ch in enumerate(self.__signal_data):
            if (paras['mask_ch'] == i).any() or not (self.__map == i).any():
                continue
            name.append(self.__signal_chs[i]['custom_channel_name'])
            ch_f, ch_pxx = self.spectrogram(ch, model='psd', nperseg=paras['nperseg'] * self.sample_rate)
            ch_pss_s.append(ch_pxx)
        ch_pss_s = np.array(ch_pss_s)
        ch_pss_std = np.std(ch_pss_s, axis=0)
        ch_pss_s = np.mean(ch_pss_s, axis=0)
        if isinstance(paras['save_path'], str):
            df = pd.DataFrame(data=np.array([ch_pss_s, ch_pss_std, ch_f]).T, columns=['psd', 'std', 'frequency'])
            df.to_csv(paras['save_path'] + '/psd_data.csv')

    # ==================== 内部工具 ====================

    def __map_check(self):
        if self.__map.size == 0:
            raise ValueError('No map. Call update_map() first.')
        return True

    def get_distance(self, a, b):
        """计算两电极间物理距离（mm）"""
        if self.__map.shape[0] == 0 or self.__distance <= 0:
            raise ValueError('need map: self.update_map(new_map, new_distance)')
        the_map = self.__map
        point_distance = self.__distance
        pos1 = np.where(the_map == a)
        pos2 = np.where(the_map == b)
        dis = np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2) * point_distance
        return dis[0]


# ==================== 多进程辅助函数 ====================

def count_spatial_resolution_coherence(__data0, __data1, __distance, __i, __j, inner_spatial_resolution,
                                       freq_band, fs, threading_lock, mean_time=[], segment_number=0):
    time0 = time.time()
    mean_coh = computation.signal_mean_coherence_version_20241014(
        __data0, __data1, freq_band=freq_band, fs=fs, nperseg=fs)
    with threading_lock:
        inner_spatial_resolution.append(np.array([__distance, mean_coh, __i, __j, segment_number]))
        mean_time.append(time.time() - time0)


def one_segment_spatial_resolution_coherence(segment_number, ch_number, inner_data_segmentation, inner_distance,
                                             freq_band, fs, spatial_resolution):
    threading_list = []
    mean_time = []
    inner_spatial_resolution = []
    threading_lock = threading.RLock()
    for inner_i in range(ch_number):
        for inner_j in range(inner_i):
            inner_p = threading.Thread(target=count_spatial_resolution_coherence,
                                       args=(inner_data_segmentation[inner_i], inner_data_segmentation[inner_j],
                                             inner_distance[inner_i][inner_j], inner_i, inner_j,
                                             inner_spatial_resolution, freq_band, fs, threading_lock,
                                             mean_time, segment_number))
            threading_list.append(inner_p)
    for inner_p in threading_list:
        inner_p.start()
    for one_threading in threading_list:
        one_threading.join()
    spatial_resolution.append(inner_spatial_resolution)


def count_spatial_resolution_correlation(__data0, __data1, __distance, __i, __j, inner_spatial_resolution,
                                         threading_lock, mean_time=[], segment_number=0):
    time0 = time.time()
    __cor, __p = scipy.stats.pearsonr(__data0, __data1)
    with threading_lock:
        inner_spatial_resolution.append(np.array([__distance, __cor, __p, __i, __j, segment_number]))
        mean_time.append(time.time() - time0)


def one_segment_spatial_resolution_correlation(segment_number, ch_number, inner_data_segmentation, inner_distance,
                                               spatial_resolution):
    threading_list = []
    mean_time = []
    inner_spatial_resolution = []
    threading_lock = threading.RLock()
    for inner_i in range(ch_number):
        for inner_j in range(inner_i):
            inner_p = threading.Thread(target=count_spatial_resolution_correlation,
                                       args=(inner_data_segmentation[inner_i], inner_data_segmentation[inner_j],
                                             inner_distance[inner_i][inner_j], inner_i, inner_j,
                                             inner_spatial_resolution, threading_lock, mean_time, segment_number))
            threading_list.append(inner_p)
    for inner_p in threading_list:
        inner_p.start()
    for one_threading in threading_list:
        one_threading.join()
    spatial_resolution.append(inner_spatial_resolution)